In [8]:
import pandas as pd

file_path = r"D:\1015 model DB\NewYorkconstruction_summary.csv"
df = pd.read_csv(file_path)

# 清理列名空格
df.columns = [c.strip() for c in df.columns]

# 确保数值列正确转换
num_cols = ['Floor area (m2)', 'Volume (m3)', 'Area-Nett (m2)',
            'U-Value (W/K-m2)', 'U-Value*Area (W/K)', 'Km (KJ/m2-K)',
            'Km*Area (KJ/K)', 'SROut (m2-K/W)', 'SRIn (m2-K/W)',
            'Embodied Carbon (kgCO2)', 'Equivalent CO2 (kgCO2)',
            'Cost of Surface Finish (GBP)']

for col in num_cols:
    if col in df.columns:
        # 去掉可能的空格/逗号等，然后转为浮点数
        df[col] = df[col].astype(str).str.replace(',', '').str.strip()
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 检查数值列是否有正确数据
print(df[num_cols].head())

# 按 Zone + Element 汇总
agg_dict = {
    'Area-Nett (m2)': 'sum',
    'U-Value (W/K-m2)': 'mean',
    'Embodied Carbon (kgCO2)': 'sum',
    'Equivalent CO2 (kgCO2)': 'sum',
    'Cost of Surface Finish (GBP)': 'sum'
}

df_grouped = df.groupby(['Zone', 'Element'], as_index=False).agg(agg_dict)

# Pivot，每个 Zone 一行，Element 类型作为列
df_area = df_grouped.pivot(index='Zone', columns='Element', values='Area-Nett (m2)').fillna(0)
df_u = df_grouped.pivot(index='Zone', columns='Element', values='U-Value (W/K-m2)').fillna(0)
df_carbon = df_grouped.pivot(index='Zone', columns='Element', values='Embodied Carbon (kgCO2)').fillna(0)

# 重命名列
df_area.columns = [f'{col}_Area' for col in df_area.columns]
df_u.columns = [f'{col}_UValue' for col in df_u.columns]
df_carbon.columns = [f'{col}_Carbon' for col in df_carbon.columns]

# 合并
df_zone_summary = pd.concat([df_area, df_u, df_carbon], axis=1).reset_index()

output_path = r"D:\1015 model DB\NewYorkconstruction_zone_summary.csv"
df_zone_summary.to_csv(output_path, index=False)

print("按 Zone 汇总完成，文件已保存：", output_path)


   Floor area (m2)  Volume (m3)  Area-Nett (m2)  U-Value (W/K-m2)  \
0          979.715      7837.72             NaN               NaN   
1              NaN          NaN           988.0             0.250   
2              NaN          NaN           988.0             5.016   
3              NaN          NaN           104.0             2.388   
4              NaN          NaN           104.0             2.388   

   U-Value*Area (W/K)  Km (KJ/m2-K)  Km*Area (KJ/K)  SROut (m2-K/W)  \
0           1977.7570           NaN             NaN             NaN   
1            246.9375        93.960      92832.4791            0.01   
2           4955.4859         7.767       7673.9931            0.04   
3            248.3582        11.074       1151.7376            0.13   
4            248.3582        11.074       1151.7376            0.13   

   SRIn (m2-K/W)  Embodied Carbon (kgCO2)  Equivalent CO2 (kgCO2)  \
0            NaN                      0.0                     NaN   
1           0.17    

In [11]:
import pandas as pd

# -----------------------------
# 1. 读取 DOE 原型 CSV
# -----------------------------
file_path = r"D:\1015 model DB\NewYorkconstruction_zone_summary.csv"  # 修改为你的文件路径
df = pd.read_csv(file_path)

# -----------------------------
# 2. 提取关键列
# -----------------------------
columns_needed = [
    'Zone', 'Element', 'Orientation (deg E of N)', 'Slope (deg)',
    'Area-Nett (m2)', 'U-Value (W/K-m2)', 'SHGC',
    'Heating SetPoint Temperature (C)', 'Cooling SetPoint Temperature (C)'
]

# 检查 SHGC 是否存在
for col in columns_needed:
    if col not in df.columns:
        df[col] = 0  # 没有就补 0

df_zone = df[columns_needed].copy()

# -----------------------------
# 3. 功能空间分类
# -----------------------------
def get_space_type(zone_name):
    zone_name = str(zone_name).lower()
    if 'gym' in zone_name:
        return 'Gym'
    elif 'classroom' in zone_name:
        return 'Classroom'
    elif 'lab' in zone_name:
        return 'Lab'
    elif 'office' in zone_name:
        return 'Office'
    else:
        return 'Other'

df_zone['SpaceType'] = df_zone['Zone'].apply(get_space_type)

# -----------------------------
# 4. 计算加权 U 值函数
# -----------------------------
def weighted_U(sub_df):
    total_area = sub_df['Area-Nett (m2)'].sum()
    if total_area == 0:
        return 0
    return (sub_df['U-Value (W/K-m2)'] * sub_df['Area-Nett (m2)']).sum() / total_area

# -----------------------------
# 5. 按 Zone + Element 汇总
# -----------------------------
zone_summary = df_zone.groupby(['Zone', 'Element']).apply(
    lambda x: pd.Series({
        'TotalArea': x['Area-Nett (m2)'].sum(),
        'WeightedU': weighted_U(x),
        'Orientation': x['Orientation (deg E of N)'].mode()[0] if not x['Orientation (deg E of N)'].mode().empty else 0,
        'SHGC': x['SHGC'].mean()
    })
).reset_index()

# -----------------------------
# 6. 按功能空间汇总
# -----------------------------
space_summary = df_zone.groupby('SpaceType').apply(
    lambda x: pd.Series({
        'TotalArea': x['Area-Nett (m2)'].sum(),
        'WeightedU': weighted_U(x),
        'AverageSHGC': x['SHGC'].mean()
    })
).reset_index()

# -----------------------------
# 7. 输出 CSV
# -----------------------------
zone_summary.to_csv(r"D:\1015 model DB\DOE_zone_summary_per_zone.csv", index=False)
space_summary.to_csv(r"D:\1015 model DB\DOE_zone_summary_per_space.csv", index=False)

print("Zone 和 SpaceType 汇总完成，已输出 CSV")


Zone 和 SpaceType 汇总完成，已输出 CSV


C:\Users\lenovo\AppData\Local\Temp/ipykernel_14756/4025183335.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  zone_summary = df_zone.groupby(['Zone', 'Element']).apply(
C:\Users\lenovo\AppData\Local\Temp/ipykernel_14756/4025183335.py:67: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  space_summary = df_zone.groupby('SpaceType').apply(


In [4]:
#生成新U值
import pandas as pd
import os

# === 1. 读取 CSV 文件 ===
input_path = r"D:\1015 model DB\NewYorkconstruction_zone_type.csv"
zone_summary = pd.read_csv(input_path)

# === 2. 自动匹配列名 ===
columns_map = {
    "Glazing_UValue": ["Glazing_UValue", "Glazing_U", "Window_UValue"],
    "Wall_UValue": ["Wall_UValue", "Wall_U"],
    "Roof_UValue": ["Roof_UValue", "Roof_U"],
    "Glazing_Area": ["Glazing_Area", "Window_Area"],
    "Wall_Area": ["Wall_Area"],
    "Roof_Area": ["Roof_Area"]
}

for key, options in columns_map.items():
    for opt in options:
        if opt in zone_summary.columns:
            zone_summary[key] = pd.to_numeric(zone_summary[opt], errors='coerce')
            break

# === 3. 计算窗墙比和映射尺度因子 ===
zone_summary["WWR_DOE"] = zone_summary["Glazing_Area"] / zone_summary["Wall_Area"]
WWR_BCHS = 0.30
zone_summary["S_scale"] = WWR_BCHS / zone_summary["WWR_DOE"]

# === 4. BCHS 校准差值 ===
delta_U_window = -0.684
delta_U_wall = -0.059
delta_U_roof = -0.067
delta_COP = 0.45
COP_DOE = 3.5

# === 5. 计算新值 ===
zone_summary["New_U_glazing"] = zone_summary["Glazing_UValue"] + zone_summary["S_scale"]*delta_U_window
zone_summary["New_U_wall"] = zone_summary["Wall_UValue"] + zone_summary["S_scale"]*delta_U_wall
zone_summary["New_U_roof"] = zone_summary["Roof_UValue"] + zone_summary["S_scale"]*delta_U_roof
zone_summary["New_COP"] = COP_DOE + zone_summary["S_scale"]*delta_COP

# === 6. 计算变化量 ===
zone_summary["Delta_U_glazing"] = zone_summary["New_U_glazing"] - zone_summary["Glazing_UValue"]
zone_summary["Delta_U_wall"] = zone_summary["New_U_wall"] - zone_summary["Wall_UValue"]
zone_summary["Delta_U_roof"] = zone_summary["New_U_roof"] - zone_summary["Roof_UValue"]
zone_summary["Delta_COP"] = zone_summary["New_COP"] - COP_DOE

# === 7. 输出文件 ===
output_cols = [
    "Zone",
    "WWR_DOE", "S_scale",
    "Glazing_UValue","New_U_glazing","Delta_U_glazing",
    "Wall_UValue","New_U_wall","Delta_U_wall",
    "Roof_UValue","New_U_roof","Delta_U_roof",
    "New_COP","Delta_COP"
]

output_path = r"D:\1015 model DB\DOE_scaled_parameters_by_zone.csv"
zone_summary[output_cols].to_csv(output_path, index=False)

print(f"✅ 已生成按 Zone 输出的新旧值对比文件：{output_path}")


✅ 已生成按 Zone 输出的新旧值对比文件：D:\1015 model DB\DOE_scaled_parameters_by_zone.csv
